In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from fuzzywuzzy import fuzz, process
from difflib import SequenceMatcher
pd.set_option('display.max_columns', 100)

In [2]:
today = f"{datetime.now():%Y%m%d}"
open_file = open(f'20220915_datadict.pkl', "rb")
datadict = pickle.load(open_file)
open_file.close()

In [3]:
open_file = open(f'scores.pkl', "rb")
scores = pickle.load(open_file)
open_file.close()

In [4]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [6]:
res = []
for dic in datadict:
    e_dict = {}
    for name, tup in dic.items():
        name = name.replace(' ','_').lower()
        if len(tup) == 2:
            e_dict[f'{name}_diff'] = float(tup[0]) - float(tup[1])
#             e_dict[f'{name}_sum'] = float(tup[0]) + float(tup[1])
        else:
            e_dict[name] = tup
    if 'bet' in e_dict:
        e_dict['odds_sum'] = e_dict['bet']['odds_checker_sum']
        del e_dict['bet']
    e_dict['median_odds'] = e_dict.pop('odds')
    if len(e_dict['median_odds']) > 0:
        e_dict['std_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                      [np.std(sorted(x)[1:-1]) for x in e_dict['full_odds']]))
        e_dict['75_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                      [np.percentile(sorted(x), 75) for x in e_dict['full_odds']]))
    res.append(e_dict)

C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
df = pd.DataFrame(res)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['median_odds'] != '') & (df['median_odds'] != {})]
df = df.reset_index(drop = True)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
for name in ['std','median','75']:
    r = []
    for i in df[[f'{name}_odds','o_names']].to_numpy():
        match = [getname(name, list(i[0].keys())) for name in i[1]]
        match = [i[0][key] for key in match]
        r.append(match)
    df[f'{name}_odds'] = r
df['odds_predict'] = df['median_odds'].apply(lambda x: np.argmin(x))
ordo_df = pd.DataFrame(df['median_odds'].tolist(), columns=['med_0', 'med_1', 'med_2'])
std_o = pd.DataFrame(df['std_odds'].tolist(), columns=['std_0', 'std_1', 'std_2'])
sevfiv_0 = pd.DataFrame(df['75_odds'].tolist(), columns=['75_0', '75_1', '75_2'])
df = pd.concat([df, std_o, ordo_df,sevfiv_0], axis=1)
df[['med_odds_prob_0','med_odds_prob_1','med_odds_prob_2']] = df[['med_0','med_1','med_2']].apply(lambda x: 1/x)
df[['75_odds_prob_0','75_odds_prob_1','75_odds_prob_2']] = df[['75_0','75_1','75_2']].apply(lambda x: 1/x)
df = df.drop([ 'half_time','o_names', '75_odds', 'median_odds',
             'std_odds', 'full_odds','match_name'], axis = 1)

In [8]:
df['target'] = scores

In [9]:
df['url'] = df['url'].apply(lambda x: x.split('/')[-1])
df['odds_url'] = df['odds_url'].apply(lambda x: x.split('/')[-2])

In [10]:
test = df[['url','odds_url']]
test['s'] = test[['url','odds_url']].apply(lambda x: similar(x.url,x.odds_url), axis = 1)
test = test.sort_values(by = 's')

C:\Users\idotz\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
test.head(30)

,url,odds_url,s
42,leon-vs-atlas,club-necaxa-v-club-leon,0.222222
254,bayern-m%C3%BCnchen-vs-vfb-stuttgart,augsburg-v-bayern-munich,0.366667
18,nordsj%C3%A6lland-vs-fc-k%C3%B8benhavn,randers-fc-v-aab,0.370370
326,kristiansund-vs-v%C3%A5lerenga,aalesund-v-sarpsborg,0.400000
22,cambuur-vs-az-alkmaar,fc-emmen-v-az,0.411765
319,malm%C3%B6-ff-vs-ifk-norrk%C3%B6ping,kalmar-ff-v-bk-hacken,0.421053
93,az-alkmaar-vs-nec-nijmegen,az-v-nec,0.470588
298,psv-eindhoven-vs-rkc-waalwijk,psv-v-rkc,0.473684
58,queens-park-rangers-vs-hull-city,qpr-v-hull,0.476190
76,royal-antwerp-vs-union-st.gilloise,antwerp-v-westerlo,0.538462


In [13]:
remove = [42,254,18,326,22,319,76]

In [14]:
df = df.drop(remove)
df = df.drop(['url','odds_url'], axis = 1)

In [15]:
df = df.reset_index(drop = True)
df = df[df['odds_sum'] > 1.03]

In [16]:
df.to_csv('f_eng_df.csv')